In [18]:
import torch
import numpy as np
from torch import matmul
from torch.autograd import Variable
from embeddings import embeds
from torch import nn
import torch.nn.functional as F
pth= 'blakepoems.txt'
def input_layer(word_idx,voca_size):
	x = torch.zeros(voca_size)
	x[word_idx] = 1.0
	return x

	
def train(n_epochs,lr,embedding_size):
    dataset,voca_size,(w2i,i2w) = embeds(pth,4)
    #it s our linear transformations, let's say 1 hidden layer
    W1 = Variable( torch.rand(embedding_size,voca_size).float(), requires_grad=True)
    #it s our second hidden layer, 
    W2 = Variable( torch.rand(voca_size,embedding_size).float(), requires_grad=True)
    for epoch in range(n_epochs):
        print(f'running {epoch+1} out of {n_epochs} epochs')
        loss_val = 0
        for i,(data, target) in enumerate(dataset):

            x = Variable(input_layer(data,voca_size)).float()
            y_true = Variable(torch.from_numpy(np.array([target])).long())

            #we are trainsforming x from (voca_size x 1) to (embedding x 1)
            z1 = matmul(W1,x)
            #it s transforming from (embedding x 1) to (voca_size x 1)
            z2= matmul(W2,z1)
            
            #activation
            log_softmax = F.log_softmax(z2, dim=0)
            
            #loss function expects input as N x C, or in our case 1 x voca_size
            #so we will reshape our prediction output 
            loss = F.nll_loss(log_softmax.view(1,-1), y_true)
            loss.backward()
            
            loss_val += loss.item()
            
            W1.data -= lr * W1.grad.data
            W2.data -= lr * W2.grad.data

            W1.grad.data.zero_()
            W2.grad.data.zero_()
            #we can compare visually, prediction and ground truth
            if (i+1) % 10000 == 0:
                print(f'\titeration - {i+1}')
                pred_ind = log_softmax.argmax(dim=0).item()
                print(f'\t\tcenter word: {i2w[data]}\t context pred.: {i2w[pred_ind]}\tg.truth : {i2w[y_true.item()]}')

    
        
        if epoch % 2 == 0:    
            print(f'Loss at epoch {epoch}: {loss_val/len(dataset)}')
            #let check for the last pairs pred and output 
            

            pass



In [19]:
train(20,0.03,10)

running 1 out of 20 epochs
	iteration - 10000
		center word: name	 context pred.: and	g.truth : my
	iteration - 20000
		center word: fly	 context pred.: and	g.truth : and
	iteration - 30000
		center word: a	 context pred.: the	g.truth : weakness
Loss at epoch 0: 6.183877958556433
running 2 out of 20 epochs
	iteration - 10000
		center word: name	 context pred.: and	g.truth : my
	iteration - 20000
		center word: fly	 context pred.: and	g.truth : and
	iteration - 30000
		center word: a	 context pred.: and	g.truth : weakness
running 3 out of 20 epochs
	iteration - 10000
		center word: name	 context pred.: and	g.truth : my
	iteration - 20000
		center word: fly	 context pred.: and	g.truth : and
	iteration - 30000
		center word: a	 context pred.: and	g.truth : weakness
Loss at epoch 2: 5.920687491680206
running 4 out of 20 epochs
	iteration - 10000
		center word: name	 context pred.: and	g.truth : my


KeyboardInterrupt: 